In [19]:
import os
import pandas as pd
import sqlalchemy

# Step 1: Create 'data/' folder
data_folder = 'data/'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
    print("✅ 'data/' folder created.")
else:
    print("📁 'data/' folder already exists.")


📁 'data/' folder already exists.


In [20]:
# Sample data for Store S001
data1 = {
    'Store_ID': ['S001']*3,
    'Date': ['2025-07-15']*3,
    'Product_ID': ['P001', 'P002', 'P003'],
    'Product_Name': ['Soap', 'Shampoo', 'Toothpaste'],
    'Quantity_Sold': [10, 5, 8],
    'Unit_Price': [20.0, 100.0, 35.0],
    'Discount_Percent': [5.0, 10.0, 0.0],
    'Payment_Mode': ['Cash', 'Card', 'UPI']
}

# Store S002
data2 = {
    'Store_ID': ['S002']*3,
    'Date': ['2025-07-15']*3,
    'Product_ID': ['P004', 'P005', 'P006'],
    'Product_Name': ['Notebook', 'Pen', 'Pencil'],
    'Quantity_Sold': [3, 20, 30],
    'Unit_Price': [50.0, 10.0, 5.0],
    'Discount_Percent': [0.0, 5.0, 2.0],
    'Payment_Mode': ['Wallet', 'Cash', 'Card']
}

# Store S003
data3 = {
    'Store_ID': ['S003']*2,
    'Date': ['2025-07-16', '2025-07-16'],
    'Product_ID': ['P007', 'P008'],
    'Product_Name': ['Bag', 'Shoes'],
    'Quantity_Sold': [1, 2],
    'Unit_Price': [700.0, 1200.0],
    'Discount_Percent': [15.0, 20.0],
    'Payment_Mode': ['Card', 'UPI']
}

# Save them as CSVs
pd.DataFrame(data1).to_csv('data/store_S001_day1.csv', index=False)
pd.DataFrame(data2).to_csv('data/store_S002_day1.csv', index=False)
pd.DataFrame(data3).to_csv('data/store_S003_day2.csv', index=False)

print("✅ Sample CSV files created.")


✅ Sample CSV files created.


In [21]:
# Read all CSVs in data/ folder
csv_files = [os.path.join('data', f) for f in os.listdir('data') if f.endswith('.csv')]
df_list = [pd.read_csv(file) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)

print(f"✅ Loaded {len(csv_files)} CSV files.")
df.head()

✅ Loaded 3 CSV files.


,Store_ID,Date,Product_ID,Product_Name,Quantity_Sold,Unit_Price,Discount_Percent,Payment_Mode
0,S001,2025-07-15,P001,Soap,10,20.0,5.0,Cash
1,S001,2025-07-15,P002,Shampoo,5,100.0,10.0,Card
2,S001,2025-07-15,P003,Toothpaste,8,35.0,0.0,UPI
3,S002,2025-07-15,P004,Notebook,3,50.0,0.0,Wallet
4,S002,2025-07-15,P005,Pen,20,10.0,5.0,Cash


In [22]:
# Drop missing values (you can also use fillna if needed)
df.dropna(inplace=True)

# Convert column names to lowercase
df.columns = df.columns.str.lower()

# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Remove duplicates based on Store_ID, Date, Product_ID
df.drop_duplicates(subset=['store_id', 'date', 'product_id'], inplace=True)

# Create new column for Total_Sale_Value
df['total_sale_value'] = df['quantity_sold'] * df['unit_price'] * (1 - df['discount_percent'] / 100)

# Categorize sales using NumPy
import numpy as np
df['sales_category'] = np.where(df['total_sale_value'] >= 10000, 'High',
                         np.where(df['total_sale_value'] >= 5000, 'Medium', 'Low'))

df.head()

,store_id,date,product_id,product_name,quantity_sold,unit_price,discount_percent,payment_mode,total_sale_value,sales_category
0,S001,2025-07-15,P001,Soap,10,20.0,5.0,Cash,190.0,Low
1,S001,2025-07-15,P002,Shampoo,5,100.0,10.0,Card,450.0,Low
2,S001,2025-07-15,P003,Toothpaste,8,35.0,0.0,UPI,280.0,Low
3,S002,2025-07-15,P004,Notebook,3,50.0,0.0,Wallet,150.0,Low
4,S002,2025-07-15,P005,Pen,20,10.0,5.0,Cash,190.0,Low


In [26]:
from sqlalchemy import create_engine

# Replace with your MySQL credentials
user = 'root'
password = 'root'
host = 'localhost'
database = 'retail_db'

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# Save to MySQL
df.to_sql('retail_sales', con=engine, if_exists='append', index=False, method='multi')
print("✅ Data inserted into MySQL table.")

✅ Data inserted into MySQL table.


In [27]:
# Total sales per store
store_sales = df.groupby('store_id')['total_sale_value'].sum().reset_index()

# Top 5 products
top_5_products = df.groupby('product_name')['total_sale_value'].sum().nlargest(5).reset_index()

# Daily sales per store
daily_sales = df.groupby(['store_id', 'date'])['total_sale_value'].sum().reset_index()


In [28]:
store_sales.to_csv('store_sales_summary.csv', index=False)
top_5_products.to_csv('top_5_products.csv', index=False)
daily_sales.to_csv('daily_sales_trend.csv', index=False)

print("✅ Reports exported as CSV files.")


✅ Reports exported as CSV files.
